## **Developing tool for detecting and counting unique people/characters in a movie clip**

In [3]:
# # for multi-object tracking algorithm
# ! pip install deepsort 
# # for object detection
# ! pip install opencv-python 

In [1]:
import cv2 as cv
import numpy as np
import deepsort
import time

In [2]:
net = cv.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
# net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
# net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [3]:
img = cv.imread('imgs/1.jpg')
cv.imshow('win', img)
cv.waitKey(1)

-1

In [4]:
ln = net.getLayerNames()
print(len(ln), ln)

# construct a blob from the image
blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
r = blob[0, 0, :, :]

cv.imshow('blob', r)
text = f'Blob shape={blob.shape}'
cv.displayOverlay('blob', text)
cv.waitKey(1)

net.setInput(blob)
t0 = time.time()
outputs = net.forward(ln)
t = time.time()

cv.displayOverlay('window', f'forward propagation time={t-t0}')
cv.imshow('window',  img)
cv.waitKey(0)
cv.destroyAllWindows()

254 ('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut_36', 'conv_37', 'bn_37', 'le

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1199: error: (-213:The function/feature is not implemented) The library is compiled without QT support in function 'cv::displayOverlay'


: 

AttributeError: module 'deepsort' has no attribute 'Tracker'

In [2]:
net = cv.dnn.readNet("yolov3.cfg", "yolov3.weights") # Load YOLO model

tracker = deepsort.Tracker()

# Define RoI and crossing line
roi = (x1, y1, x2, y2)
crossing_line = (x_line, y_line)

# Load video
cap = cv2.VideoCapture("your_video.mp4")

# Initialize counters
total_count = 0
counted_ids = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Crop frame to RoI
    cropped_frame = frame[y1:y2, x1:x2]

    # Detect objects
    blob = cv2.dnn.blobFromImage(cropped_frame, 1/255, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward()

    # Get detections
    class_ids = []
    boxes = []
    confidences = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # Assuming 'person' is class 0
                center_x, center_y, width, height = int(detection[0] * cropped_frame.shape[1]), int(detection[1] * cropped_frame.shape[0]), int(detection[2] * cropped_frame.shape[1]), int(detection[3] * cropped_frame.shape[0])
                x, y = int(center_x - width / 2), int(center_y - height / 2)
                boxes.append([x, y, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Track objects
    tracks = tracker.update(np.array(boxes), np.array(confidences), np.array(class_ids))

    # Count objects that cross the line
    for track in tracks:
        if track.is_confirmed() and track.get_centroid()[1] > crossing_line[1] and track.track_id not in counted_ids:
            total_count += 1
            counted_ids.add(track.track_id)

    # Visualize
    # ... (draw bounding boxes, crossing line, and count)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("Total count:", total_count)

error: OpenCV(4.10.0) /io/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'readNetFromDarknet'
